In [1]:
import os
%pwd

'f:\\ml_projects\\playing_card_detection\\notbooks'

In [2]:
os.chdir("../")
%pwd

'f:\\ml_projects\\playing_card_detection'

In [3]:
from dataclasses import dataclass
from pathlib import Path

@dataclass(frozen=True)
class ModelTrainerConfig:
    root_dir:Path
    pretrained_model_weight_name: str
    dataset_yaml_file: Path
    EPOCHS: int
    BATCH_SIZE: int

In [14]:
from Playing_card_detection.constants import *
from Playing_card_detection.utils.common import read_yaml_file,create_directories

class ConfigurationManager:
    def __init__(self,config_filepath = CONFIG_FILEPATH):
        self.config = read_yaml_file(CONFIG_FILEPATH)
        create_directories([self.config.artifacts_root])

    def get_model_trainer_config(self) -> ModelTrainerConfig:
        temp_config = self.config.model_trainer
        create_directories([temp_config.root_dir])

        model_trainer_config = ModelTrainerConfig(
            root_dir = temp_config.root_dir,
            pretrained_model_weight_name = temp_config.model_pretrained_weight_name,
            dataset_yaml_file = temp_config.dataset_yaml_file,
            EPOCHS = temp_config.EPOCHS,
            BATCH_SIZE = temp_config.BATCH_SIZE
        )

        return model_trainer_config

In [21]:
from Playing_card_detection.utils.exception import CustomException
from Playing_card_detection.utils.logger import logger
from Playing_card_detection.utils.common import read_for_yolo,save_yaml,read_yaml_file
from Playing_card_detection.constants import *

class ModelTrainer:
    def __init__(self,config:ModelTrainerConfig,config_filepath = CONFIG_FILEPATH):
        self.config = config
        self.data_ingestion_config = read_yaml_file(config_filepath).data_ingestion

    def __clone_repository__(self):
        os.system("git clone https://github.com/ultralytics/yolov5.git")

    def __get_num_classes__(self):
        nc = read_for_yolo(self.config.dataset_yaml_file,check = "nc")
        return nc
    
    def __custom_yaml_for_yolo__(self,model_name,nc):
        yolo_config = read_for_yolo(f"yolov5/models/{model_name}.yaml")
        yolo_config['nc'] = nc

        save_yaml(f"yolov5/models/custom_{model_name}.yaml",yolo_config)
        logger.info("Successfully saved Custom Yolo Config")
    
    def train(self):
        num_classes = self.__get_num_classes__()
        print(num_classes)
        logger.info("Got the Number of Classes")

        if not "yolov5" in os.listdir(os.getcwd()):
            self.__clone_repository__()
            logger.info("Cloned Yolov5 Repoisitory")
        else:
            logger.info("Already Cloned Yolov5 Repoisitory")

        model_name = self.config.pretrained_model_weight_name.split(".")[0]
        self.__custom_yaml_for_yolo__(model_name = model_name,nc = num_classes)

        ##>>>>>>>>>>>>>>>>>>>>>>> Use Git Bash for following Commands <<<<<<<<<<<<<<<<<<<<<<<<<<<<<

        os.system(f"cp {self.data_ingestion_config.local_data_file} {os.getcwd()}")## Copying Zip to main directory
        os.system(f"unzip data.zip")

        os.system(f"cd yolov5/ && python train.p    y --img 416 --batch {self.config.BATCH_SIZE} --epochs {self.config.EPOCHS} --data ../data.yaml --cfg ./model/custom_{model_name}.yaml --weights {self.config.pretrained_model_weight_name} --name {model_name}_results --cache")
        logger.info("Training Completed")
        
        os.system(f"cp yolov5/runs/train/{model_name}_results/weights/best.pt yolov5/") ## Copy best.pt to yolov5 directory
        os.system(f"cp yolov5/runs/train/{model_name}_results/weights/best.pt {self.config.root_dir}")
        logger.info("Copied Weights To Artifacts")

        os.system("rm -rf test")
        os.system("rm -rf train")
        os.system("rm -rf valid")
        os.system("rm -rf data.yaml")
        os.system("rm -rf data.zip")
        os.system("rm -rf yolov5/runs")


In [23]:
import sys
try:
    config = ConfigurationManager()
    model_trainer_config = config.get_model_trainer_config()
    model_trainer = ModelTrainer(model_trainer_config)
    model_trainer.train()
except Exception as e:
    raise CustomException(e,sys)

52
[ 2024-01-12 10:13:06,714 ] - 28 - 4095685097 - Got the Number of Classes
[ 2024-01-12 10:13:06,716 ] - 34 - 4095685097 - Already Cloned Yolov5 Repoisitory


[ 2024-01-12 10:13:06,832 ] - 23 - 4095685097 - Successfully saved Custom Yolo Config
[ 2024-01-12 10:13:51,494 ] - 45 - 4095685097 - Training Completed
